In [30]:
%load_ext autoreload
%autoreload 2

import random

import pandas as pd

from titanic import TitanicHelper


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
helper = TitanicHelper("./data/passengers.parquet")

In [10]:
helper.did_they_survive("Braund, Mr. Owen Harris")

False

In [11]:
train = pd.read_csv("./data/train.csv")
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


Prediction is just matching a name exactly to the "lookup" table

In [18]:
train["prediction"] = train["Name"].apply(helper.did_they_survive).astype("Int64")

In [20]:
train["prediction"].value_counts(dropna=False)

prediction
<NA>    422
0       343
1       126
Name: count, dtype: Int64

How many mistakes do we make if we successfully matched people by name?

In [23]:
predictions = train[train["prediction"].notna()].copy()
(predictions["prediction"] == predictions["Survived"]).value_counts()

True     460
False      9
Name: count, dtype: Int64

Who did we "predict" incorrectly?

In [24]:
predictions[predictions["prediction"] != predictions["Survived"]]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,prediction
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0
84,85,1,2,"Ilett, Miss. Bertha",female,17.0,0,0,SO/C 14885,10.5000,NaN,S,0
216,217,1,3,"Honkanen, Miss. Eliina",female,27.0,0,0,STON/O2. 3101283,7.9250,NaN,S,0
289,290,1,3,"Connolly, Miss. Kate",female,22.0,0,0,370373,7.7500,NaN,Q,0
605,606,0,3,"Lindell, Mr. Edvard Bengtsson",male,36.0,1,0,349910,15.5500,NaN,S,1
620,621,0,3,"Yasbeck, Mr. Antoni",male,27.0,1,0,2659,14.4542,NaN,C,1
651,652,1,2,"Doling, Miss. Elsie",female,18.0,0,1,231919,23.0000,NaN,S,0
793,794,0,1,"Hoyt, Mr. William Fisher",male,NaN,0,0,PC 17600,30.6958,NaN,C,1
828,829,1,3,"McCormack, Mr. Thomas Joseph",male,NaN,0,0,367228,7.7500,NaN,Q,0


Turns out even getting an exact match can be wrong because Kaggle and the Titanic website disagree!

Let's try matching the test set

In [38]:
test = pd.read_csv("./data/test.csv")
test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [39]:
test["Survived"] = test["Name"].apply(helper.did_they_survive).astype("Int64")
test["Survived"].value_counts(dropna=False)

Survived
<NA>    202
0       162
1        54
Name: count, dtype: Int64

For a first submission let's fill in the NA values probabilistically based on the survived vs. died ratio in the training data

In [40]:
train["Survived"].value_counts(normalize=True)

Survived
0    0.616162
1    0.383838
Name: proportion, dtype: float64

In [41]:
def get_random_survival(N=1):
    """return 0 or 1 with a 60-40 probability"""
    return [random.choice([0] * 6 + [1] * 4) for _ in range(N)]

In [42]:
na_size = len(test[test["Survived"].isna()])
test.loc[test["Survived"].isnull(), "Survived"] = get_random_survival(na_size)
test["Survived"].value_counts(dropna=False)

Survived
0    267
1    151
Name: count, dtype: Int64

In [43]:
test[["PassengerId", "Survived"]].to_csv("./data/test_submission_1.csv", index=False)